# Kaggle competition 1

## I) Creating hypothesis for relationships within data, visualizing the data, understanding and describing the data

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm

### 1) Creating a DataFrame of the Data, visualizing contingency tables

In [2]:
# Load data
data_train = np.load('data_train.npy', allow_pickle = True)
data_test = np.load('data_test.npy', allow_pickle = True)
vocab_map = np.load('vocab_map.npy', allow_pickle=True)

# Load labels_train from CSV and extract the 'label' column
labels_train_df = pd.read_csv('label_train.csv')  # Assuming this is your labels file
labels_train = labels_train_df['label'].values  # Extract the labels as a NumPy array

# Convert training data to a DataFrame for visualization
df_train = pd.DataFrame(data_train)

# Add column names using vocab_map
df_train.columns = vocab_map

# Add the target labels to the DataFrame
df_train['label'] = labels_train

# Select a few important terms for creating contingency tables
selected_terms = vocab_map[:1]  # Let's say we are analyzing the first 5 terms

# Create contingency tables for each selected term
for term in selected_terms:
    # Convert term counts to binary (presence/absence)
    df_train[term + '_present'] = df_train[term].apply(lambda x: 1 if x > 0 else 0)
    
    # Generate a contingency table for the term
    contingency_table = pd.crosstab(df_train[term + '_present'], df_train['label'])
    
    print(f"Contingency table for term: {term}")
    print(contingency_table)
    print("\n")

print('''
Interpretation:

Label 0: There are 7,123 documents where the term "00" is absent (not present) and the label is 0.
Label 1: There are 2,297 documents where the term "00" is absent and the label is 1.

Label 0: There is 1 document where the term "00" is present and the label is 0.
Label 1: There is 1 document where the term "00" is present and the label is 1.
''')

Contingency table for term: 00
label          0     1
00_present            
0           7123  2297
1              1     1



Interpretation:

Label 0: There are 7,123 documents where the term "00" is absent (not present) and the label is 0.
Label 1: There are 2,297 documents where the term "00" is absent and the label is 1.

Label 0: There is 1 document where the term "00" is present and the label is 0.
Label 1: There is 1 document where the term "00" is present and the label is 1.



### 2) Transform the data with TF-IDF and visualize the DataFrame

In [4]:
tfidf_transformer = TfidfTransformer()
data_train_tfidf = tfidf_transformer.fit_transform(data_train)
data_test_tfidf = tfidf_transformer.transform(data_test)

# Convert the sparse matrix to a dense format (if needed)
data_train_tfidf_dense = data_train_tfidf.toarray()

# Create a DataFrame for easier visualization, with terms as column names
df_tfidf = pd.DataFrame(data_train_tfidf_dense, columns=vocab_map)

In [5]:
# Display the first few rows of the DataFrame
print(df_tfidf.shape)

(9422, 26354)


## II) Dimensionalty reduction, and model selection

### 1) Dimensionality reduction using truncated SVD, model selection using F1-score and fitting a simple logistic regression classifier

In [7]:
# Define the range of components to test (broad range from 50 to 1500 with steps of 100)
component_range = list(range(500, 1501, 100))  # Test from 500 to 1500 with steps of 100

# Initialize lists to store results
f1_scores = []

# Define the scorer for macro F1 score
f1_scorer = make_scorer(f1_score, average='macro')

# Loop over different values for n_components with tqdm to show progress
for n_components in tqdm(component_range, desc="Testing SVD Components"):
    # Apply Truncated SVD with n_components
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    X_reduced = svd.fit_transform(data_train_tfidf)
    
    # Initialize Logistic Regression
    clf = LogisticRegression(max_iter=1000, random_state=42)
    
    # Perform 5-fold cross-validation and calculate the average macro F1 score
    scores = cross_val_score(clf, X_reduced, labels_train, cv=5, scoring=f1_scorer)
    f1_scores.append(scores.mean())
    
    # Print the result for this number of components
    print(f"n_components={n_components}, Macro F1 Score={scores.mean()}")

# After loop, find the best number of components
best_n_components = component_range[np.argmax(f1_scores)]
print(f"Best number of components: {best_n_components}")
print(f"Best Macro F1 score: {max(f1_scores)}")

Testing SVD Components:   9%|█████▎                                                     | 1/11 [00:10<01:41, 10.19s/it]

n_components=500, Macro F1 Score=0.6504907660796119


Testing SVD Components:  18%|██████████▋                                                | 2/11 [00:34<02:45, 18.40s/it]

n_components=600, Macro F1 Score=0.6535555040606469


Testing SVD Components:  27%|████████████████                                           | 3/11 [00:58<02:48, 21.11s/it]

...

n_components=1300, Macro F1 Score=0.6503540873568958


Testing SVD Components:  91%|████████████████████████████████████████████████████▋     | 10/11 [06:52<00:56, 56.74s/it]

n_components=1400, Macro F1 Score=0.6488226545575827


Testing SVD Components: 100%|██████████████████████████████████████████████████████████| 11/11 [07:55<00:00, 43.22s/it]

n_components=1500, Macro F1 Score=0.6508758865792623
Best number of components: 600
Best Macro F1 score: 0.6535555040606469


### 2) Keeping the 3rd best model (avoid over-fitting)

In [9]:
# Apply Truncated SVD with 600 components
svd_best = TruncatedSVD(n_components=1200, random_state=42)
X_reduced_best = svd_best.fit_transform(data_train_tfidf)

### 3) Fitting different learning algorithms

In [12]:
# Define the SVM model
svm_clf = LinearSVC(max_iter=1000, random_state=42)

# Perform 5-fold cross-validation for macro F1 score
svm_f1_scores = cross_val_score(svm_clf, X_reduced_best, labels_train, cv=5, scoring=f1_scorer)

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (SVM): {svm_f1_scores.mean()}")

Cross-validated Macro F1 Score (SVM): 0.694617681179871


In [14]:
# Apply Truncated SVD with the raw count data
svd = TruncatedSVD(n_components=1200, random_state=42)
X_reduced_best_count = svd.fit_transform(data_train)  # Using raw count data without TF-IDF

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "D:\miniCondaa\envs\kaggle1IFT3395\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\miniCondaa\envs\kaggle1IFT3395\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "D:\miniCondaa\envs\kaggle1IFT3395\lib\site-packages\sklearn\naive_bayes.py", line 759, in fit
    self._count(X, Y)
  File "D:\miniCondaa\envs\kaggle1IFT3395\lib\site-packages\sklearn\naive_bayes.py", line 881, in _count
    check_non_negative(X, "MultinomialNB (input X)")
  File "D:\miniCondaa\envs\kaggle1IFT3

In [17]:
# Set negative values to zero
X_reduced_best_count[X_reduced_best_count < 0] = 0

# Initialize Multinomial Naive Bayes
nb_clf = MultinomialNB()

# Perform 5-fold cross-validation for macro F1 score
nb_f1_scores = cross_val_score(nb_clf, data_train, labels_train, cv=5, scoring=make_scorer(f1_score, average='macro'))

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (Naive Bayes on counts): {nb_f1_scores.mean()}")

Cross-validated Macro F1 Score (Naive Bayes on counts): 0.7104751581201816


In [18]:
# Initialize Logistic Regression
log_clf = LogisticRegression(max_iter=1000, random_state=42)

# Perform 5-fold cross-validation for macro F1 score
log_f1_scores = cross_val_score(log_clf, X_reduced_best, labels_train, cv=5, scoring=make_scorer(f1_score, average='macro'))

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (Logistic Regression): {log_f1_scores.mean()}")

Cross-validated Macro F1 Score (Logistic Regression): 0.6516869522987132


In [19]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)

# Perform 5-fold cross-validation for macro F1 score
rf_f1_scores = cross_val_score(rf_clf, X_reduced_best, labels_train, cv=5, scoring=make_scorer(f1_score, average='macro'))

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (Random Forest): {rf_f1_scores.mean()}")


Cross-validated Macro F1 Score (Random Forest): 0.43412083789333844


In [20]:
import xgboost as xgb

# Initialize XGBoost
xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=6, random_state=42)

# Perform 5-fold cross-validation for macro F1 score
xgb_f1_scores = cross_val_score(xgb_clf, X_reduced_best, labels_train, cv=5, scoring=make_scorer(f1_score, average='macro'))

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (XGBoost): {xgb_f1_scores.mean()}")


Cross-validated Macro F1 Score (XGBoost): 0.6363559427423845


In [21]:
from sklearn.neural_network import MLPClassifier

# Initialize MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Perform 5-fold cross-validation for macro F1 score
mlp_f1_scores = cross_val_score(mlp_clf, X_reduced_best, labels_train, cv=5, scoring=make_scorer(f1_score, average='macro'))

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (MLP Classifier): {mlp_f1_scores.mean()}")


Cross-validated Macro F1 Score (MLP Classifier): 0.6850856935489185


In [22]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)

# Perform 5-fold cross-validation for macro F1 score
knn_f1_scores = cross_val_score(knn_clf, X_reduced_best, labels_train, cv=5, scoring=make_scorer(f1_score, average='macro'))

# Print the average macro F1 score across the 5 folds
print(f"Cross-validated Macro F1 Score (KNN): {knn_f1_scores.mean()}")

Cross-validated Macro F1 Score (KNN): 0.5468534858136735
